In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from IPython.display import display
from pathlib import Path

In [2]:
#Excel reading function
df_dict = pd.read_excel(r"C:\Users\mattp\Finance Project\Concatenate and Excel\All Assets Final Before Analysis.xlsx",  sheet_name = None)
df_AllAssets = df_dict['All Assets']
del df_dict['All Assets']
display(df_dict.keys())
df_AllAssets.columns = df_AllAssets.columns.to_series().replace('Unnamed:\s\d+',np.nan,regex=True).ffill().values

dict_keys(['Stock- NVIDIA', 'Stock- Amazon', 'Stock- Apple', 'Stock- Microsoft', 'Stock- Tesla', 'Mutual Fund- FNF', 'Mutual Fund-FSTP', 'Bond- 13 Week TBill', 'Bond- 5YTBY', 'Bond- 10 YTBill Yield', 'Commodity- Copper', 'Commodity- Oil', 'Commodity- Gold', 'Commodity- Steel', 'Credit Data- SNP Futures', 'FX Pair- USD EUR', 'FX Pair- USD JPY', 'Index Fund- S&P'])

In [3]:
def analysis(df_common):
    starting_point = df_common[df_common.Open>0].iloc[0]
    date_starting_point = starting_point[0]
    df_common["Starting Point"] = date_starting_point
    
    series_PCT_common = df_common["Day to Day"]
    series_PCT_SNP = df_SNP["Day to Day"]

    df_common['Date'] = pd.to_datetime(df_common['Date'], errors='coerce')
    
    df_common["Day to Day"] = df_common["Close"].pct_change()
    df_common['Month to Month'] = df_common.groupby(df_common['Date'].dt.to_period('M'))['Day to Day'].transform('mean')
    df_common['Year to Year'] = df_common.groupby(df_common['Date'].dt.to_period('Y'))['Day to Day'].transform('mean')

    df_common_DTD = df_common["Day to Day"].iloc[1:]
    trading_days = 252
    df_common["Volatility"] = df_common_DTD.rolling(window = trading_days).std()*np.sqrt(trading_days)

    df_common['Date'] = df_common['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    return df_common

In [4]:
dict_postAnalysis = {}
for key in df_dict:
    df_common = df_dict[key]
    df = analysis(df_common)
    dict_postAnalysis[key] = df

C:\Users\mattp\AppData\Local\Temp\ipykernel_10784\3571236038.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_common['Month to Month'] = df_common.groupby(df_common['Date'].dt.to_period('M'))['Day to Day'].transform('mean')
C:\Users\mattp\AppData\Local\Temp\ipykernel_10784\3571236038.py:10: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_common['Year to Year'] = df_common.groupby(df_common['Date'].dt.to_period('Y'))['Day to Day'].transform('mean')
C:\Users\mattp\AppData\Local\Temp\ipykernel_10784\3571236038.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_common['Month to Month'] = df_common.groupby(df_common['Date'].dt.to_period('M'))['Day to Day'].transform('mean')
C:\Users\mattp\AppData\Local\Temp\ipykernel_10784\3571236038.py:10: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_co

In [5]:
list_dfNames = ["Stock- NVIDIA", "Stock- Amazon", "Stock- Apple", "Stock- Microsoft", "Stock- Tesla", "Mutual Fund- FNF", "Mutual Fund-FSTP", "Bond- 13 Week TBill", "Bond- 5YTBY", "Bond- 10 YTBill Yield", "Commodity- Copper", "Commodity- Oil", "Commodity- Gold", "Commodity- Steel", "Credit Data- SNP Futures", "FX Pair- USD EUR", "FX Pair- USD JPY", "Index Fund- S&P"]
df_Data_Concat = pd.concat(dict_postAnalysis.values(), axis = 1, keys=list_dfNames[:])

In [6]:
writer = pd.ExcelWriter(r"C:\Users\mattp\Finance Project\Concatenate and Excel\All Assets Before Correlation.xlsx")
df_AllAssets.to_excel(writer, sheet_name="All Assets")
for key in dict_postAnalysis:
    new_dataframe = df_Data_Concat[key]
    new_dataframe.to_excel(writer, sheet_name=key)
writer.close()